In [ ]:
import pandas as pd

## Creating data
There are two core objects in pandas: the DataFrame and the Series.



### DataFrame
A DataFrame is a table. It contains an array of individual entries, each of which has a certain value. Each entry corresponds to a row (or record) and a column.
- We use the pd.DataFrame() constructor to generate DataFrame objects. The syntax for declaring a new one is a dictionary whose keys are the column names and whose values are a list of entries. This is the standard way of constructing a new DataFrame.
- The dictionary-list constructor assigns values to the column labels, but just uses an ascending count from 0 (0, 1, 2, 3, ...) for the row labels. Sometimes this is OK, but oftentimes we will want to assign these labels ourselves.
- The list of row labels used in a DataFrame is known as an Index. We can assign values to it by using an index parameter in constructor.

In [ ]:
pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'], 
              'Sue': ['Pretty good.', 'Bland.']},
             index=['Product A', 'Product B'])

### Series
A Series, by contrast, is a sequence of data values. If a DataFrame is a table, a Series is a list.
- A Series is, in essence, a single column of a DataFrame. So row labels can be assigned to the Series the same way as before, using an index parameter. However, a Series does not have a column name, it only has one overall name.

In [ ]:
pd.Series([30, 35, 40], index=['2015 Sales', '2016 Sales', '2017 Sales'], name='Product A')

## Reading data files
- `pd.read_csv()` function to read the data into a DataFrame. To make pandas use a column for the index (instead of creating a new one from scratch), we can specify an `index_col`.
- use the `shape` attribute to check how large the resulting DataFrame is.
- examine the contents of the resultant DataFrame using the `head()` command, which grabs the first five rows.

## Native accessors
Native Python objects provide good ways of indexing data. Pandas carries all of these over, which helps make it easy to start with.
- we can access the property of an object by accessing it as an attribute. [dataframe.column]
- we can access its values using the indexing ([]) operator.
- Neither of them is more or less syntactically valid than the other, but the indexing operator [] does have the advantage that it can handle column names with reserved characters (like whitespace) in them.
- to drill down to a single specific value, we need only use the indexing operator [] once more

## Indexing in pandas
Pandas indexing works in one of two paradigms using its own accessor operators, `loc` and `iloc`. Both loc and iloc are row-first, column-second. This is the opposite of what we do in native Python, which is column-first, row-second.

### Index-based selection
selecting data based on its numerical position in the data. `iloc` follows this paradigm.
- On its own, the `:` operator, which also comes from native Python, means "everything". When combined with other selectors, however, it can be used to indicate a range of values.
- It's also possible to pass a list. (A slice of a list is also a list.)
- negative numbers can be used in selection. This will start counting forwards from the end of the values.

### Label-based selection
The second paradigm for attribute selection is the one followed by the loc operator: label-based selection. In this paradigm, it's the data index value, not its position, which matters.

> `iloc` is conceptually simpler than `loc` because it ignores the dataset's indices. When we use `iloc` we treat the dataset like a big matrix (a list of lists), one that we have to index into by position. `loc`, by contrast, uses the information in the indices to do its work. Since your dataset usually has meaningful indices, it's usually easier to do things using `loc` instead.

### Choosing between loc and iloc
When choosing or transitioning between loc and iloc, there is one "gotcha" worth keeping in mind, which is that the two methods use slightly different indexing schemes.

iloc uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded. So 0:10 will select entries 0,...,9. loc, meanwhile, indexes inclusively. So 0:10 will select entries 0,...,10.

Why the change? Remember that loc can index any stdlib type: strings, for example. If we have a DataFrame with index values Apples, ..., Potatoes, ..., and we want to select "all the alphabetical fruit choices between Apples and Potatoes", then it's a lot more convenient to index df.loc['Apples':'Potatoes'] than it is to index something like df.loc['Apples', 'Potatoet'] (t coming after s in the alphabet).

This is particularly confusing when the DataFrame index is a simple numerical list, e.g. 0,...,1000. In this case df.iloc[0:1000] will return 1000 entries, while df.loc[0:1000] return 1001 of them! To get 1000 elements using loc, you will need to go one lower and ask for df.loc[0:999].

Otherwise, the semantics of using loc are the same as those for iloc.

### Manipulating the index
Label-based selection derives its power from the labels in the index. Critically, the index we use is not immutable. We can manipulate the index in any way we see fit.

The `set_index()` method can be used to do the job. This is useful if you can come up with an index for the dataset which is better than the current one.

### Conditional selection
Conditional operation on a column produces a Series of True/False booleans based on the values of each record. This result can then be used inside of `loc` to select the relevant data. 
- We can use logical operator, the ampersand (&) or a pipe (|) to bring the two conditional operations together and then use inside of `loc` to select the relevant data.

Pandas comes with a few built-in conditional selectors, two of which we will highlight here.
- The first is `isin`. `isin` lets you select data whose value "is in" a list of values.
- The second is `isnull` (and its companion `notnull`). These methods let you highlight values which are (or are not) empty (NaN).

### Assigning data
Assigning data to a DataFrame is easy. You can assign either a constant value or with an iterable of values.

### Summary functions
Pandas provides many simple "summary functions" (not an official name) which restructure the data in some useful way.
- `describe()` method generates a high-level summary of the attributes of the given column. It is type-aware, meaning that its output changes based on the data type of the input.
- to get some particular simple summary statistic about a column in a DataFrame or a Series, there is usually a helpful pandas function that makes it happen:
    - to see the mean of the points allotted, we can use the `mean()` function
    - To see a list of unique values we can use the `unique()` function
    - To see a list of unique values and how often they occur in the dataset, we can use the `value_counts()` method

### Maps
A map is a term, borrowed from mathematics, for a function that takes one set of values and "maps" them to another set of values. In data science we often have a need for creating new representations from existing data, or for transforming data from the format it is in now to the format that we want it to be in later. Maps are what handle this work, making them extremely important for getting your work done!

There are two mapping methods that you will use often.

`map()` is the first, and slightly simpler one. The function you pass to `map()` should expect a single value from the Series (a point value), and return a transformed version of that value. `map()` returns a new Series where all the values have been transformed by your function.

`apply()` is the equivalent method if we want to transform a whole DataFrame by calling a custom method on each row.

In [ ]:
review_points_mean = reviews.points.mean()
reviews.points.map(lambda p: p - review_points_mean)

def remean_points(row):
    row.points = row.points - review_points_mean
    return row

reviews.apply(remean_points, axis='columns')

If we had called `reviews.apply()` with `axis='index'`, then instead of passing a function to transform each row, we would need to give a function to transform each column.

Note that `map()` and `apply()` return new, transformed Series and DataFrames, respectively. They don't modify the original data they're called on. If we look at the first row of reviews, we can see that it still has its original points value.

Pandas provides many common mapping operations as built-ins.  
- For an operation with a lot of values on the left-hand side and a single value on the right-hand side, Pandas perform the operation on each of the values on the left-hand side with the single value on the right-hand side.
- Pandas will also understand what to do if we perform these operations between Series of equal length.

These operators are faster than `map()` or `apply()` because they use speed ups built into pandas. All of the standard Python operators (`>`, `<`, `==`, and so on) work in this manner.

However, they are not as flexible as `map()` or `apply()`, which can do more advanced things, like applying conditional logic, which cannot be done with addition and subtraction alone.

### Groupwise analysis
each group generated by `groupby()` is a slice of a DataFrame containing only data with values that match. This DataFrame is accessible directly using the `apply()` method, and we can then manipulate the data in any way we see fit.  
We can also use any of the summary functions with this data. This will generate the summary of the selected column for each group like, mean, min, count etc.

For even more fine-grained control, you can also group by more than one column.

Another groupby() method worth mentioning is agg(), which lets you run a bunch of different functions on your DataFrame simultaneously. For example, we can generate a simple statistical summary of the dataset as follows:

In [ ]:
reviews.groupby(['country']).price.agg([len, min, max])

### Multi-indexes
depending on the operation we run, `groupby()` will sometimes result in what is called a multi-index. A multi-index differs from a regular index in that it has multiple levels. Multi-indices have several methods for dealing with their tiered structure which are absent for single-level indices. They also require two levels of labels to retrieve a value.

Generally the multi-index method most often used is the one for converting back to a regular index, the `reset_index()` method. It resets the index for the new dataframe.

### Sorting
When outputting the result of a `groupby`, the order of the rows is dependent on the values in the index, not in the data.

- To get data in the order want it in we can sort it ourselves. The `sort_values()` method is handy for this.  
- `sort_values()` defaults to an ascending sort, where the lowest values go first. However, most of the time we want a descending sort, where the higher numbers go first. 
- To sort by index values, use the companion method `sort_index()`. This method has the same arguments and default order.
- You can also sort by more than one column at a time.

In [ ]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_values(by='len', ascending=False)

countries_reviewed.sort_values(by=['country', 'len'])

### Dtypes
The data type for a column in a DataFrame or a Series is known as the `dtype`.
- Use the `dtype` property to grab the type of a specific column. Use it on columns.
- The `dtypes` property returns the dtype of every column in the DataFrame. Use it on dataframes.
- A DataFrame or Series index has its own dtype, too.

Data types tell about how pandas is storing the data internally. `float64` means that it's using a 64-bit floating point number; `int64` means a similarly sized integer instead, and so on.

One peculiarity to keep in mind is that columns consisting entirely of strings do not get their own type; they are instead given the `object` type.

It's possible to convert a column of one type into another wherever such a conversion makes sense by using the `astype()` function.

### Missing data
Entries missing values are given the value `NaN`, short for "Not a Number". For technical reasons these NaN values are always of the `float64` dtype.

Pandas provides some methods specific to missing data. To select `NaN` entries you can use `pd.isnull()` (or its companion `pd.notnull()`).

Pandas provides a really handy method for replacing missing values: `fillna()`. `fillna()` provides a few different strategies for mitigating such data. For example, 
- we can simply replace each `NaN` with an "Unknown".
- Or we could fill each missing value with the first non-null value that appears sometime after the given record in the database. This is known as the backfill strategy.

Alternatively, we may have a non-null value that we would like to replace. One way to reflect this in the dataset is using the `replace()` method.The `replace()` method is handy for replacing missing data which is given some kind of sentinel value in the dataset: things like `"Unknown"`, `"Undisclosed"`, `"Invalid"`, and so on.

### Renaming
`rename()` lets you change index names and/or column names. It lets you rename index or column values by specifying a `index` or `column` keyword parameter, respectively. It supports a variety of input formats, but usually a Python dictionary is the most convenient.  
Both the row index and the column index can have their own name attribute. The complimentary `rename_axis()` method may be used to change these names.

In [ ]:
reviews.rename(columns={'points': 'score'})

reviews.rename(index={0: 'firstEntry', 1: 'secondEntry'})

reviews.rename_axis("wines", axis='rows').rename_axis("fields", axis='columns')

### Combining
When performing operations on a dataset, we will sometimes need to combine different DataFrames and/or Series in non-trivial ways. Pandas has three core methods for doing this. In order of increasing complexity, these are `concat()`, `join()`, and `merge()`. Most of what `merge()` can do can also be done more simply with `join()`.

The simplest combining method is `concat()`. Given a list of elements, this function will smush those elements together along an axis. This is useful when we have data in different DataFrame or Series objects but having the same fields (columns).

The middlemost combiner in terms of complexity is `join()`. `join()` lets you combine different DataFrame objects which have an index in common.

In [ ]:
canadian_youtube = pd.read_csv("../input/youtube-new/CAvideos.csv")
british_youtube = pd.read_csv("../input/youtube-new/GBvideos.csv")

pd.concat([canadian_youtube, british_youtube])


left = canadian_youtube.set_index(['title', 'trending_date'])
right = british_youtube.set_index(['title', 'trending_date'])

left.join(right, lsuffix='_CAN', rsuffix='_UK')

The `lsuffix` and `rsuffix` parameters are necessary here because the data has the same column names in both British and Canadian datasets. If this wasn't true (because, say, we'd renamed them beforehand) we wouldn't need them.